In [1]:
import cv2
import numpy as np
from sklearn.metrics import pairwise

In [2]:
def calibrate (gray,alpha):
    global bg_img
    if bg_img is None:
        bg_img=gray.copy().astype(np.float64)
        return
    cv2.accumulateWeighted(gray,bg_img,alpha)

In [3]:
def hand_cont(gray):
    global bg_img
    diff=cv2.absdiff(bg_img.astype(np.uint8),gray)
    thresh=cv2.threshold(diff,25,255,cv2.THRESH_BINARY)[1]
    cnts,hier=cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if len(cnts)==0:
        return
    else:
        cont=max(cnts,key=cv2.contourArea)
        return (thresh,cont)

In [4]:
def gest_count(thresh,cont):
    con_hull=cv2.convexHull(cont)
    ext_left=tuple(con_hull[con_hull[:,:,0].argmin()][0])
    ext_right=tuple(con_hull[con_hull[:,:,0].argmax()][0])
    ext_top=tuple(con_hull[con_hull[:,:,1].argmin()][0])
    ext_bottom=tuple(con_hull[con_hull[:,:,1].argmax()][0])
    cx=int((ext_left[0]+ext_right[0])/2)
    cy=int((ext_top[1]+ext_bottom[1])/2)
    cy+=(cy*0.15)
    cy=int(cy)
    dist=pairwise.euclidean_distances([(cx,cy)],Y=[ext_left,ext_right,ext_top,ext_bottom])[0]
    max_dis=dist[dist.argmax()]
    r=int(0.8*max_dis)
    circum=2*np.pi*r
    cir_roi=np.zeros(thresh.shape[:2],dtype='uint8')
    cv2.circle(cir_roi,(cx,cy),r,255,1)
    cir_roi=cv2.bitwise_and(thresh,thresh,mask=cir_roi)
    cnts,hier=cv2.findContours(cir_roi,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    fin=0
    for c in cnts:
        (x,y,w,h)=cv2.boundingRect(c)
        if ((cy+cy*0.25)>(y+h)) and ((circum*0.25)>c.shape[0]):
            fin+=1
    return fin

    

In [5]:
def img_resize(img,width=None,height=None,inter=cv2.INTER_AREA):
    dim=None
    (h,w)=img.shape[:2]
    if width is None and height is None:
        return
    if width is None:
        r=height/float(h)
        dim=(int(w*r),height)
    else:
        r=width/float(w)
        dim=(width,int(h*r))
    return cv2.resize(img,dim,interpolation=inter)

In [6]:
alpha=0.5
bg_img=None
t,r,b,l=[10,300,250,600]
no_frames=0
gesture=None
val=[]
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    frame=img_resize(frame,width=700)
    frame=cv2.flip(frame,1)
    clone=frame.copy()
    (h,w)=frame.shape[:2]
    roi=frame[t:b,r:l]
    gray=cv2.cvtColor(roi,cv2.COLOR_RGB2GRAY)
    gray=cv2.GaussianBlur(gray,(7,7),0)
    if no_frames<20:
        calibrate(gray,alpha)
    else:
        hand=hand_cont(gray)
        if hand is not None:
            (thresh,cont)=hand
            cv2.drawContours(clone,[cont+(r,t)],-1,(0,255,0),3)
            fin=gest_count(thresh,cont)
            cv2.putText(clone,str(fin),(50,54),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),3,cv2.LINE_AA)
    cv2.rectangle(clone,(l,t),(r,b),(0,255,0),3)
    no_frames+=1
    cv2.imshow('Output',clone)
    if cv2.waitKey(1)&0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()